# Data Consistency Checks

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
df_prods= pd.read_csv(r'C:\Users\willi\5-2023 Instacart Basket Analysis\Data\Original Data\4.3_orders_products\products.csv', index_col = False)

In [3]:
df_ords= pd.read_csv(r'C:\Users\willi\5-2023 Instacart Basket Analysis\Data\Prepared Data\orders_wrangled.csv', index_col = False)

In [4]:
#1 Consistency checks
df_prods.describe()

,product_id,aisle_id,department_id,prices
count,49693.000000,49693.000000,49693.000000,49693.000000
mean,24844.345139,67.770249,11.728433,9.994136
std,14343.717401,38.316774,5.850282,453.519686
min,1.000000,1.000000,1.000000,1.000000
25%,12423.000000,35.000000,7.000000,4.100000
50%,24845.000000,69.000000,13.000000,7.100000
75%,37265.000000,100.000000,17.000000,11.200000
max,49688.000000,134.000000,21.000000,99999.000000


In [5]:
#2 Consistency checks
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,order_dow,time_ordered,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


The max number of "time_ordered" is 2.3. The number should be higher, either 12 or 24 depending on what "time" you're using. Also, the max number of "days_since_prior_order" is 3, while the 50 percentile is 7. How is the max lower than the 50 percentile? "Order_number"'s max and min are the same while their percentiles are higher.

In [8]:
#3 Check for mixed-types
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

In [9]:
#4
df_ords.dtypes

Unnamed: 0                  int64
order_id                    int64
user_id                     int64
eval_set                   object
order_number                int64
order_dow                   int64
time_ordered                int64
days_since_prior_order    float64
dtype: object

In [10]:
#5 Run a check for missing values
df_ords.isnull().sum()

Unnamed: 0                     0
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
time_ordered                   0
days_since_prior_order    206209
dtype: int64

"days_since_prior_order" has many missing values. If subsequent orders took place on the same day, the number 0 might be entered and come up as a missing value. Or those users only made one order making their "days_since_prior_order" null.

In [ ]:
#6 Address missing values

I considered dropping the column using the "df_ords.dropna(subset =[days_since_prior_order], inplace = True)". However, the information is still valuable. I decided to leave it as it is.

In [11]:
#7 Checking for duplicates
df_dups = df_ords[df_ords.duplicated()]

In [12]:
df_dups

,Unnamed: 0,order_id,user_id,eval_set,order_number,order_dow,time_ordered,days_since_prior_order


In [14]:
df_dups.shape

(0, 8)

No duplicates

In [ ]:
#8 Addressing duplicates

Because there were no duplicates, no action was taken. If there were, I would have used this code "df_ords_no_dups = df_ords.drop_duplicates()"

In [15]:
#9 Export
path=r'C:\Users\willi\5-2023 Instacart Basket Analysis'

In [16]:
df_ords.to_csv(os.path.join(path, 'Data','Prepared Data', 'orders_checked.csv'))

In [18]:
df_prods.to_csv(os.path.join(path, 'Data','Prepared Data', 'products_checked.csv'))